In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%%capture
%pwd

In [3]:
%%capture
cd ..

In [4]:
PATH=%pwd

In [5]:
%%capture
PATH

In [6]:
from random import sample 
from src.funcs.utils import *

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv('PP_API_KEY')
insttoken=os.getenv('PP_INSTTOKEN')


In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Pharmapendium Safety Query

#### Query for Available Data Fields

In [9]:
parameters = dict()
parameters['apiKey'] = api_key
parameters['insttoken'] = insttoken
url_safety_fields = os.getenv('URL_SAFETY_FIELDS')

In [10]:
df_fields=get_data_fields(url_safety_fields, parameters)

In [11]:
df_fields

,
0,specie
1,dose
2,route
3,smiles
4,document
5,effect
6,documentYear
7,doseType
8,source
9,id


#### Query for Available Facet Fields

In [123]:
parameters = dict()
parameters['apiKey'] = api_key
parameters['insttoken'] = insttoken
url_facets = os.getenv('URL_SAFETY_FACET_FIELDS')


In [13]:
df_facets=get_facet_fields(url_facets, parameters)

In [14]:
df_facets

,facet_fields
0,drugs
1,species
2,sources
3,effects
4,route
5,doseType
6,documentYear


#### Query for All Clinical Drugs

In [124]:
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters["limitation.count"] = 0
parameters["facets"]="drugs"
parameters['specie']='human'
parameters['leaf']=True
url_safety=os.getenv('URL_SAFETY_SEARCH')

In [125]:
with requests.Session() as s:
    r = s.get(url_safety, params=parameters)
r.status_code 

200

In [126]:
drugs_df=get_drugs(url_safety, parameters)

In [128]:
drugs_df.info(); drugs_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4176 entries, 0 to 4175
Data columns (total 1 columns):
drug    4176 non-null object
dtypes: object(1)
memory usage: 32.8+ KB


,drug
0,1-13C-Caprylic Acid
1,5-Methoxypsoralen
2,Abacavir Sulfate
3,Abacavir Sulfate; Dolutegravir Sodium; Lamivudine
4,Abacavir Sulfate; Lamivudine
5,Abacavir Sulfate; Lamivudine; Zidovudine
6,Abaloparatide
7,Abametapir
8,Abarelix
9,Abatacept


In [19]:
drugs_df.to_pickle(f'{PATH}/data/raw/clinical_drugs.pkl')

#### Query for Pre-Marketing Adverse Events

In [20]:
drugs_df=pd.read_pickle(f'{PATH}/data/raw/clinical_drugs.pkl')

In [21]:
drugs_df.info(); drugs_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4176 entries, 0 to 4175
Data columns (total 1 columns):
drug    4176 non-null object
dtypes: object(1)
memory usage: 32.8+ KB


,drug
0,1-13C-Caprylic Acid
1,5-Methoxypsoralen
2,Abacavir Sulfate
3,Abacavir Sulfate; Dolutegravir Sodium; Lamivudine
4,Abacavir Sulfate; Lamivudine


In [22]:
drug_list=list(drugs_df.drug)
len(drug_list)
drug_list=drug_list[:5]

4176

In [23]:
parameters = dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters['limitation.count']=0
parameters['species']='human'
parameters['leaf']=True
parameters['facets'] = 'effects'
url_safety = os.getenv('URL_SAFETY_SEARCH')

In [24]:
df_aes=get_drug_ae(url_safety, drug_list, parameters)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.61it/s]


In [25]:
df_aes.info(); df_aes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 2 columns):
drug_name    601 non-null object
ae           600 non-null object
dtypes: object(2)
memory usage: 9.5+ KB


,drug_name,ae
0,1-13C-Caprylic Acid,None
1,5-Methoxypsoralen,acanthoma
2,5-Methoxypsoralen,acne
3,5-Methoxypsoralen,acute generalised exanthematous pustulosis
4,5-Methoxypsoralen,acute leukaemia


### Pharmapendium PK Query

In [129]:
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters["limitation.count"] = 0
parameters["facets"]="drugs"
parameters['specie']='human'
parameters['leaf']=True
url_pk=os.getenv('URL_PK_SEARCH')

In [27]:
with requests.Session() as s:
    r = s.get(url_safety, params=parameters)
r.status_code 

200

In [28]:
pk_drugs_df=get_drugs(url_safety, parameters)

In [29]:
pk_drugs_df.info(); pk_drugs_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4176 entries, 0 to 4175
Data columns (total 1 columns):
drug    4176 non-null object
dtypes: object(1)
memory usage: 32.8+ KB


,drug
0,1-13C-Caprylic Acid
1,5-Methoxypsoralen
2,Abacavir Sulfate
3,Abacavir Sulfate; Dolutegravir Sodium; Lamivudine
4,Abacavir Sulfate; Lamivudine


#### Query for Available Fields

In [30]:
parameters = dict()

parameters['apiKey'] = api_key
parameters['insttoken'] = insttoken
url_pk=os.getenv('URL_PK_FIELDS')

In [31]:
df_fields=get_data_fields(url_pk, parameters)

In [32]:
df_fields

,
0,specie
1,studyName
2,smiles
3,document
4,studyGroup
5,documentYear
6,source
7,sampleSize
8,metabolitesEnantiomers
9,drug


#### Query for all PK parameters

In [33]:
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters['limitation.count']=0
parameters['species']='human'
parameters['facets']='parameters'
parameters['leaf']=True
url_pk=os.getenv('URL_PK_SEARCH')

In [34]:
df_params=get_pk_params(url_pk, parameters)


In [35]:
df_params.info(); df_params

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 1 columns):
pk_parameters    99 non-null object
dtypes: object(1)
memory usage: 920.0+ bytes


,pk_parameters
0,%absorbed
1,Ae
2,Air excretion
3,Alpha
4,Amount absorbed
...,...
94,Vd/F
95,Vss
96,Vss/F
97,Vz


#### Query for PK Data

In [36]:
drugs_df=pd.read_pickle(f'{PATH}/data/raw/clinical_drugs.pkl')

In [37]:
drugs_df.head()

,drug
0,1-13C-Caprylic Acid
1,5-Methoxypsoralen
2,Abacavir Sulfate
3,Abacavir Sulfate; Dolutegravir Sodium; Lamivudine
4,Abacavir Sulfate; Lamivudine


In [47]:
pk_param_list=['AUC', 'T1/2', 'Cmax', 'Vd', 'CL', 'Plasma protein binding', 'Bioavailability']

field_list=['drug', 'parameter', 'sampleSize', 'sex', 'dose', 'duration', 'route', 'comments','valueNormalized', 'unitNormalized', 'valueOriginal']

drugList=list(drugs_df.drug)

drugList=drugList[:5]
# drugList

In [48]:
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters["limitation.count"] = 0
# parameters["facets"]="drugs"
parameters['species']='human'
# parameters['leaf']=True
url_pk=os.getenv('URL_PK_SEARCH')

In [49]:
# test counts vs. GUI
hit_count_list=[]
url=os.getenv('URL_PK_SEARCH')
with requests.Session() as s:
    for drug in tqdm.tqdm(drugList):
        parameters["drugs"] = drug
        parameters['limitation.count']=0
        #     parameters['sources']= 'FDA Approval Packages', 'EMA approval documents'
        r = s.get(url, params=parameters, stream=False).json()
        hits = r['data']['countTotal']
        hit_count_list.append(int(hits))
        

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.28it/s]


In [50]:
hit_count_list

[0, 0, 1180, 178, 100]

In [62]:
pk_param_list=['AUC', 'T1/2', 'Cmax']
field_list=['drug', 'parameter', 'sampleSize', 'sex', 'dose']


In [63]:
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters['species']='human'
url_pk=os.getenv('URL_PK_SEARCH')

In [64]:
df=get_PK_data(url_pk, drugList, field_list,  pk_param_list,  parameters)




  0%|                                                                                            | 0/5 [00:00<?, ?it/s]


 20%|████████████████▊                                                                   | 1/5 [00:00<00:02,  1.51it/s]


 40%|█████████████████████████████████▌                                                  | 2/5 [00:01<00:01,  1.75it/s]


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:10<00:06,  3.18s/it]


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:11<00:02,  2.54s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:12<00:00,  2.48s/it]


In [65]:
df.info(); df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 5 columns):
drug          776 non-null object
parameter     776 non-null object
sampleSize    615 non-null float64
sex           8 non-null object
dose          776 non-null object
dtypes: float64(1), object(4)
memory usage: 30.4+ KB


,drug,parameter,sampleSize,sex,dose
0,Abacavir Sulfate,Cmax,12.0,None,600 mg
1,Abacavir Sulfate,AUC,12.0,None,150 mg
2,Abacavir Sulfate,Cmax,18.0,None,16 mg/kg
3,Abacavir Sulfate,Cmax,15.0,None,300-600 mg
4,Abacavir Sulfate,Cmax,2.0,None,300 mg
...,...,...,...,...,...
771,Abacavir Sulfate; Lamivudine,Cmax,25.0,None,600/300 mg
772,Abacavir Sulfate; Lamivudine,Cmax,25.0,None,600 mg/300 mg
773,Abacavir Sulfate; Lamivudine,T1/2,25.0,None,600/300 mg
774,Abacavir Sulfate; Lamivudine,AUC,25.0,None,600/300 mg


In [27]:
df.to_csv(f'{PATH}/data/raw/pp_pk_drugs.csv', index=False)

### Pharmapendium FAERS Query

#### Query for all data fields

In [10]:
parameters=dict()
parameters['apiKey']=api_key
parameters['insttoken']=insttoken
url_faers_fields=os.getenv('URL_FAERS_FIELDS')

In [12]:
df_fields=get_data_fields(url_faers_fields, parameters)

In [13]:
df_fields

,
0,sendDate
1,manufacturerName
2,reportedIndications
3,primarySuspectDrugs
4,reportType
5,weightNormalized
6,ageUnit
7,outcomes
8,manufacturerNumber
9,reporterOccupation


### Query for Available Facets Fields

In [14]:
parameters=dict()
parameters['apiKey']=api_key
parameters['insttoken']=insttoken
url_faers_facets=os.getenv('URL_FAERS_listFacets')

In [15]:
df_facets=get_facet_fields(url_faers_facets, parameters)

In [16]:
df_facets

,facet_fields
0,drugs
1,secondarySuspectDrugsAndClasses
2,concomitantDrugsAndClasses
3,interactingDrugsAndClasses
4,effects
5,sources
6,secondarySuspectDrugs
7,concomitantDrugs
8,interactingDrugs
9,sex


#### Query for All Drugs in FAERS

In [33]:
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters['limitation.count'] = 0
parameters['facets']="drugs"
parameters['leaf']=True


url_faers=os.getenv('URL_FAERS')

In [34]:
with requests.Session() as s:
    r = s.get(url_faers, params=parameters)
r.status_code 

200

In [35]:
drugs_df=get_drugs(url_faers, parameters)

In [36]:
drugs_df.info(); drugs_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2954 entries, 0 to 2953
Data columns (total 1 columns):
drug    2954 non-null object
dtypes: object(1)
memory usage: 23.2+ KB


,drug
0,Abacavir Sulfate
1,Abacavir Sulfate; Dolutegravir Sodium; Lamivudine
2,Abacavir Sulfate; Lamivudine
3,Abacavir Sulfate; Lamivudine; Zidovudine
4,Abaloparatide


In [23]:
drugs_df.to_pickle(f'{PATH}/data/raw/faers_drugs.pkl')

##### Query for Specific Drug

In [37]:
drugs_df['drug']=drugs_df['drug'].apply(lambda x: str.lower(x))

In [38]:
drugs_df.head()

,drug
0,abacavir sulfate
1,abacavir sulfate; dolutegravir sodium; lamivudine
2,abacavir sulfate; lamivudine
3,abacavir sulfate; lamivudine; zidovudine
4,abaloparatide


In [28]:
drugs_df[drugs_df['drug'].str.contains('ribo', regex=True)]

,drug
106,alpha-tocopherol acetate; ascorbic acid; bioti...
241,ascorbic acid; biotin; cholecalciferol; cyanoc...
242,ascorbic acid; biotin; cyanocobalamin; dexpant...
243,ascorbic acid; biotin; cyanocobalamin; dexpant...
244,ascorbic acid; biotin; cyanocobalamin; dexpant...
245,ascorbic acid; biotin; cyanocobalamin; dexpant...
246,ascorbic acid; biotin; cyanocobalamin; dexpant...
247,ascorbic acid; cyanocobalamin; biotin; folic a...
636,chloramphenicol; desoxyribonuclease; fibrinolysin
2436,ribociclib succinate


#### Query for Post-marketing Drug AEs

In [9]:
drugs_df=pd.read_pickle(f'{PATH}/data/raw/faers_drugs.pkl')

In [10]:
drug_list=list(drugs_df.drug)
len(drug_list)
drug_list[:5]

2940

['Abacavir Sulfate',
 'Abacavir Sulfate; Dolutegravir Sodium; Lamivudine',
 'Abacavir Sulfate; Lamivudine',
 'Abacavir Sulfate; Lamivudine; Zidovudine',
 'Abaloparatide']

In [11]:
drug_sample=sample(drug_list, 2)
drug_sample

['Omadacycline Tosylate', 'Sodium Propionate']

In [39]:
drug_sample=['Ribociclib Succinate']

In [52]:
parameters=dict()
parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters['limitation.count'] = 0  
parameters['facets']="effects"
parameters['leaf']=True
# parameters['leaf']=False


url_faers=os.getenv('URL_FAERS')

In [53]:
df_aes_faers=get_drug_aes_faers(url_faers, drug_sample, parameters)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

1977


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.29it/s]


In [54]:
df_aes_faers.info(); df_aes_faers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977 entries, 0 to 1976
Data columns (total 3 columns):
drug     1977 non-null object
ae       1977 non-null object
count    1977 non-null int64
dtypes: int64(1), object(2)
memory usage: 46.5+ KB


,drug,ae,count
0,Ribociclib Succinate,Abdominal discomfort,101
1,Ribociclib Succinate,Abdominal distension,67
2,Ribociclib Succinate,Abdominal hernia,1
3,Ribociclib Succinate,Abdominal hernia infection,1
4,Ribociclib Succinate,Abdominal injury,1


In [17]:
df_aes_faers.ae.value_counts()

Toxic epidermal necrolysis                1
Multiple organ dysfunction syndrome       1
Blood creatine phosphokinase increased    1
Neutropenia                               1
Pemphigoid                                1
Mouth haemorrhage                         1
Oral mucosal exfoliation                  1
Pseudomonas infection                     1
Blood chloride decreased                  1
Dermatitis                                1
Anxiety                                   1
Dermatitis exfoliative                    1
Blood lactate dehydrogenase increased     1
Rash                                      1
Rash macular                              1
Photophobia                               1
Pathogen resistance                       1
Mucosal erosion                           1
Pyrexia                                   1
Dermatitis bullous                        1
Eyelid disorder                           1
Conjunctivitis                            1
Malaise                         

### Chemistry Query

#### Query for Available Data Fields

In [9]:
url_chem_fields=os.getenv('URL_CHEM_FIELDS')
url_chem_facets_fields=os.getenv('URL_CHEM_FACET_FIELDS')
url_chem_post_search=os.getenv('URL_CHEM_SEARCH')

In [11]:
parameters=dict()
parameters['apiKey']=api_key
parameters['insttoken']=insttoken
url_chem_fields=os.getenv('URL_CHEM_FIELDS')

In [13]:
df_fields=get_data_fields(url_chem_fields, parameters)

In [14]:
df_fields

,
0,moleculeFile
1,smiles
2,name
3,formula
4,metabolism
5,molecularWeight


#### Query for Available Facet Fields

In [18]:
parameters=dict()
parameters['apiKey']=api_key
parameters['insttoken']=insttoken
url_chem_facets=os.getenv('URL_CHEM_FACET_FIELDS')

In [16]:
df_facets=get_facet_fields(url_chem_facets, parameters)

In [17]:
df_facets

,facet_fields
0,drugs


#### Chem Structure Search

In [114]:
payload=dict()
# parameters=dict()
payload['apiKey']=api_key
payload['insttoken']=insttoken
url_chem_post_search=os.getenv('URL_CHEM_SEARCH')


In [115]:
url_chem_post_search='https://api-dev.ppnp.cm-elsevier.com/v1/chemistry/search'

In [ ]:
payload['query']={ 
    "structure": "\n  Ketcher 04272117552D 1   1.00000     0.00000     0\n\n  6  6  0     0  0            999 V2000\n  -12.2731    4.5825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  -11.4071    4.0825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  -11.4071    3.0825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  -12.2731    2.5825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  -13.1392    3.0825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  -13.1392    4.0825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  1  2  1  0     0  0\n  2  3  2  0     0  0\n  3  4  1  0     0  0\n  4  5  2  0     0  0\n  5  6  1  0     0  0\n  6  1  2  0     0  0\nM  END\n",
    "type": "Substructure"}

In [117]:
payload

{'apiKey': 'e091cb8e7cbbff3706c5db121fccb53c',
 'insttoken': 'bb1804dcaa3a8dcf07ad5334d4a67c75',
 'query': {'structure': '\n  Ketcher 04272117552D 1   1.00000     0.00000     0\n\n  6  6  0     0  0            999 V2000\n  -12.2731    4.5825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  -11.4071    4.0825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  -11.4071    3.0825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  -12.2731    2.5825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  -13.1392    3.0825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  -13.1392    4.0825    0.0000 C   0  0  0  0  0  0  0        0  0  0\n  1  2  1  0     0  0\n  2  3  2  0     0  0\n  3  4  1  0     0  0\n  4  5  2  0     0  0\n  5  6  1  0     0  0\n  6  1  2  0     0  0\nM  END\n',
  'type': 'Substructure'}}

In [118]:
r = requests.post(url_chem_post_search, payload)

In [119]:
print(r.request.url)
print(r.text)
print(r.request.body)
print(r.request.headers)
print(r.status_code)

https://api-dev.ppnp.cm-elsevier.com/v1/chemistry/search

apiKey=e091cb8e7cbbff3706c5db121fccb53c&insttoken=bb1804dcaa3a8dcf07ad5334d4a67c75&query=structure&query=type
{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '109', 'Content-Type': 'application/x-www-form-urlencoded'}
415


### Efficacy

#### Query for Available Data Fields

In [12]:
parameters = dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
url_efficacy_fields = os.getenv('URL_EFFICACY_FIELDS')

In [13]:
url_efficacy_fields

'https://api.elsevier.com/pharma/efficacy/listDataFields'

In [14]:
df_fields=get_data_fields(url_efficacy_fields, parameters)

In [15]:
df_fields

,
0,specie
1,treatment
2,smiles
3,document
4,comparativeGroup
5,studyDesign
6,monoCombination
7,documentYear
8,source
9,sampleSize


#### Query for Available Facet Fields

In [16]:
parameters = dict()
parameters['apiKey'] = api_key
parameters['insttoken'] = insttoken
url_efficacy_facets = os.getenv('URL_EFFICACY_FACET_FIELDS')


In [17]:
url_efficacy_facets

'https://api.elsevier.com/pharma/efficacy/listFacets'

In [18]:
df_facets=get_facet_fields(url_efficacy_facets, parameters)

In [19]:
df_facets

,facet_fields
0,drugs
1,species
2,sources
3,indications
4,endpoints
5,pathogens
6,route
7,monoCombination
8,sampleSize
9,phase


#### Query for Indication

In [130]:
parameters = dict()
parameters['apiKey'] = api_key
parameters['insttoken'] = insttoken
parameters['drugs']='Abacavir Sulfate'
parameters['limitation.count']=0
parameters['species']='human'
parameters['facets'] = ['indications', 'species']
parameters['leaf']=True
url_efficacy_search=os.getenv('URL_EFFICACY_SEARCH')

In [131]:
url_efficacy_search

'https://api.elsevier.com/pharma/efficacy/search'

In [132]:
with requests.Session() as s:
        r = s.get(url_efficacy_search, params=parameters).json()

In [133]:
# r
r['data']
r['facets']
# r['facets']['indications']['children']


{'countTotal': 2981, 'countLimited': 0, 'items': []}

{'indications': {'children': [{'data': {'id': '2',
     'name': 'HIV',
     'count': 290}},
   {'data': {'id': '10',
     'name': 'HIV infected with CD4+ cell counts 100 cells/mm3',
     'count': 56}},
   {'data': {'id': '34', 'name': 'HIV-1 and HIV-2 infection', 'count': 2}},
   {'data': {'id': '4', 'name': 'HIV-1 infected', 'count': 148}},
   {'data': {'id': '11',
     'name': 'HIV-1 infected patients with AIDS Dementia Complex',
     'count': 52}},
   {'data': {'id': '1', 'name': 'HIV-1 infection', 'count': 1550}},
   {'data': {'id': '9',
     'name': 'HIV-1 infection in antiretoviral (ART)-naive patients',
     'count': 65}},
   {'data': {'id': '5',
     'name': 'HIV-1 infection in antiretroviral (ART)-experienced (>12 weeks previous therapy; ZDV, 3TC or other therapy in the last 6 months) children',
     'count': 136}},
   {'data': {'id': '14',
     'name': 'HIV-1 infection in antiretroviral (ART)-experienced but TDF naive patients, receiving a stable regimen containing 3 NRTIs, o

In [82]:
lst=r['facets']['indications']['children']
lst[0]
len(lst)

{'data': {'id': 'HhD8XsFrmzB',
  'name': 'Infections and infestations',
  'isLeaf': False,
  'count': 2981},
 'children': [{'data': {'id': 'wdw2Ncv3zD9',
    'name': 'Viral infectious disorders',
    'parentID': 'HhD8XsFrmzB',
    'isLeaf': False,
    'count': 2981},
   'children': [{'data': {'id': 'fDSfXlXXvd-',
      'name': 'Retroviral infections',
      'parentID': 'wdw2Ncv3zD9',
      'isLeaf': False,
      'count': 2981},
     'children': [{'data': {'id': 'a8cRj3dQjdH',
        'name': 'HIV infection',
        'parentID': 'fDSfXlXXvd-',
        'isLeaf': False,
        'count': 2929},
       'children': [{'data': {'id': 'RCHgLRO1C0F',
          'name': 'HIV-1 infection',
          'parentID': 'a8cRj3dQjdH',
          'isLeaf': True,
          'count': 2583}},
        {'data': {'id': 'gEkhkj0EnP8',
          'name': 'Infection human immunodeficiency virus',
          'parentID': 'a8cRj3dQjdH',
          'isLeaf': True,
          'count': 346}}]},
      {'data': {'id': 'Z1ob-Alt1rB

1

In [46]:
lst[0]

{'data': {'id': 'RDr-FwfaFAE',
  'name': 'Blood and lymphatic system disorders',
  'isLeaf': False,
  'count': 34064},
 'children': [{'data': {'id': 'KNgYqq758c_',
    'name': 'Anaemias nonhaemolytic and marrow depression',
    'parentID': 'RDr-FwfaFAE',
    'isLeaf': False,
    'count': 13779},
   'children': [{'data': {'id': 'kjQaDsqUKV5',
      'name': 'Anaemia deficiencies',
      'parentID': 'KNgYqq758c_',
      'isLeaf': False,
      'count': 2157},
     'children': [{'data': {'id': 'SWIzA8t6mS-',
        'name': 'Iron deficiency anaemia',
        'parentID': 'kjQaDsqUKV5',
        'isLeaf': False,
        'count': 2157},
       'children': [{'data': {'id': 'wk__vxECrRG',
          'name': 'Anemia iron-deficiency',
          'parentID': 'SWIzA8t6mS-',
          'isLeaf': True,
          'count': 857}},
        {'data': {'id': 'w2nEIoPoBDD',
          'name': 'Anemia iron-deficiency in hemodialysis patients',
          'parentID': 'SWIzA8t6mS-',
          'isLeaf': True,
         